# Test the main_minibatch.py

In [1]:
from __future__ import absolute_import
import time
import numpy as np
import dataset
from model import RNN
# use_gpu(-1) # -1:cpu; 0,1,2,..: gpu


def define_tensor_size(prompt_id):
    tensor_ranges = {
        1: (50, 50),
        2: (50, 50),
        3: (50, 30),
        4: (50, 30),
        5: (50, 30),
        6: (50, 30),
        7: (50, 50),
        8: (50, 100)
    }
    return tensor_ranges[prompt_id]
e = 0.01
lr = 0.1
drop_rate = 0.
prompt_id = 1
vocab_size = 0  # 0 is define to automated infer vocab-size
sent_len, doc_len = define_tensor_size(prompt_id)  # sent_len is batch_size of tensor
hidden_size = [500]
word_embedding_size = 200
# try: gru, lstm
cell = "gru"
# try: sgd, momentum, rmsprop, adagrad, adadelta, adam, nesterov_momentum
optimizer = "nesterov_momentum"
train_path, dev_path, test_path = './data/fold_0/train.tsv', './data/fold_0/dev.tsv', './data/fold_0/test.tsv'
(train_x, train_masks, train_y), (dev_x, dev_masks, dev_y), (test_x, test_masks, test_y), vocab, vocab_size =\
    dataset.get_data((train_path, dev_path, test_path), prompt_id, vocab_size, doc_len, sent_len)

print "#word size = ", vocab_size
print "#train_x shape = ", train_x[0].shape

print "training..."

Using Theano backend.


#word size =  5516
#train_x shape =  (50L, 50L)
training...


In [2]:
from sklearn.utils import shuffle
def batch_generator(X, masks, y):
    while(True):
        for X_, mask,y_ in zip(*shuffle(X, masks, y)):
            yield (X_, mask, y_)
train_batch = batch_generator(train_x, train_masks, train_y)
x,m,y_=train_batch.next()
print '#batch shape = ', x.shape

#batch shape =  (50L, 50L)


In [21]:
print "compiling..."
from mylayers.encoding_layer import SentEncoderLayer
from mylayers.syntax_attention import SyntaxAttentionLayer
from mylayers.utils import init_weights, init_bias
import theano
import numpy as np
import theano.tensor as T
from optimizers import *
from theano.sandbox.rng_mrg import MRG_RandomStreams as RandomStreams
# Word_Embdding layer
params=[]
rng = RandomStreams(1234)
embeddings = theano.shared(0.2 * np.random.uniform(
    -1.0, 1.0,(vocab_size, 200)).astype(theano.config.floatX),name='WEmb')  # add one for PADDING at the end
params.append(embeddings)
idxs = T.imatrix()
x=x.astype('int')
mask = T.matrix("mask")
X = embeddings[idxs].reshape((idxs.shape[0], idxs.shape[1], 200))
sent_encoder_layer = SentEncoderLayer(rng, X, 200, [200],
                                      'gru', 'hh', 0,
                                      1, doc_len, mask)
layer_input = sent_encoder_layer.activation

params+=sent_encoder_layer.params
# sent_X = layer_input[layer_input.shape[0] - 1, :]
sent_X = layer_input[layer_input.shape[0] - 1, :]

sent_mask = T.switch(mask.sum(axis=0)> 0, 1.0, 0.0)
syntax_vector = init_bias(200, 'syntax_vector')

# W_a = init_weights([2*200, 200],name='W_a')
# W_u = init_weights([200, 1],name='W_u')
# b = init_bias(200,name='b')
# params.extend([W_a, W_u, b])
# syntax_matrix = T.reshape(T.tile(syntax_vector, 50), (50,200))
# concat = T.concatenate([sent_X, syntax_matrix], axis=1)
# strength = T.dot(T.tanh(T.dot(concat, W_a)+b), W_u).flatten()
# strength_mask = strength*sent_mask
# a = T.nnet.softmax(strength_mask)
# c = T.dot(a, sent_X)
syntax_att = SyntaxAttentionLayer('2', (doc_len, sent_encoder_layer.hidden_size),
                                  sent_X, sent_mask, syntax_vector)
c = syntax_att.activation
W_out = init_weights((200,1), 'mse_W')
b_out = init_bias(1, 'mse_b')
params.append(W_out)
params.append(b_out)
y_pred = T.dot(c, W_out)+b_out
y_true = T.scalar(('y_true'))
res = T.pow(y_pred-y_true, 2).mean()
# clip = T.clip(res, 0.0, 5)
import theano
clip = theano.gradient.grad_clip(res,0.0,1.0)
gparams = []

for param in params:
    gparam = T.grad(clip, param)
    gparams.append(gparam)
optimizer = eval('sgd')
updates = optimizer(params, gparams)

f = theano.function([idxs, mask, y_true], [c,y_pred,res,clip],updates=updates, on_unused_input='ignore')
for i in range(10):
    c_, y_pred_ ,r,clip_= f(x,np.asarray(m, dtype=theano.config.floatX), y_)
    print c_.shape, y_pred_,r,clip_
    # print params

compiling...


(1L, 200L) [[-0.00576363]] 36.0691947937 36.0691947937


(1L, 200L) [[ 4.12832594]] 3.50316381454 3.50316381454


(1L, 200L) [[ 12.34467888]] 40.2549514771 40.2549514771


(1L, 200L) [[ 2.01836085]] 15.8534498215 15.8534498215


...

(1L, 200L) [[ 27.61671638]] 467.282440186 467.282440186


(1L, 200L) [[-13.13246346]] 366.051147461 366.051147461


(1L, 200L) [[ 3.25847411]] 7.51596403122 7.51596403122


(1L, 200L) [[ 4.81119967]] 1.41324627399 1.41324627399


(1L, 200L) [[ 7.05557489]] 1.11423838139 1.11423838139


In [23]:
for i in xrange(20):
    X, mask, y = train_batch.next()
    print mask.shape,X.shape
    print X
    print y

(50L, 50L) (50L, 50L)
[[  41.   44.  216. ...,    0.    0.    0.]
 [  10.   17.  109. ...,    0.    0.    0.]
 [  23.   24.  508. ...,    0.    0.    0.]
 ..., 
 [   0.    0.    0. ...,    0.    0.    0.]
 [   0.    0.    0. ...,    0.    0.    0.]
 [   0.    0.    0. ...,    0.    0.    0.]]
8.0
(50L, 50L) (50L, 50L)
[[   41.    10.    12. ...,     0.     0.     0.]
 [  110.    18.   385. ...,     0.     0.     0.]
 [    4.  1263.   451. ...,     0.     0.     0.]
 ..., 
 [    0.     0.     0. ...,     0.     0.     0.]
 [    0.     0.     0. ...,     0.     0.     0.]
 [    0.     0.     0. ...,     0.     0.     0.]]
10.0
(50L, 50L) (50L, 50L)
[[  41.   19.   19. ...,    0.    0.    0.]
 [ 110.   27.   59. ...,    0.    0.    0.]
 [   1.   33.   10. ...,    0.    0.    0.]
 ..., 
 [   0.    0.    0. ...,    0.    0.    0.]
 [   0.    0.    0. ...,    0.    0.    0.]
 [   0.    0.    0. ...,    0.    0.    0.]]
8.0
(50L, 50L) (50L, 50L)
[[  41.   19.   10. ...,    0.    0.    0.]
 [ 

In [6]:
import theano.tensor as T
import theano
import numpy as np

from utils import init_weights, init_bias
b = np.array([[0, 2], [3, 4]]).astype("float32")
a = np.array([1]).astype('float32')
m= T.fmatrices("m")
x= T.tile(m,(1,3))
y=T.switch(x.sum(axis=0)>0,1.0,0.0)
y=y[None,:]
y[...]=b
out = theano.function([m],y)
out(a)

TypeError: 'TensorVariable' object does not support item assignment

In [23]:
import numpy as np
x=np.random.randn((10)) * 0.1
x=np.tile(x,(1,2))
x=x.reshape((2,10))

print x
print x.shape

[[-0.00509187 -0.06839405 -0.00483718  0.09116028  0.02922797  0.08932855
   0.16974536  0.26184026  0.16582535  0.30151564]
 [-0.00509187 -0.06839405 -0.00483718  0.09116028  0.02922797  0.08932855
   0.16974536  0.26184026  0.16582535  0.30151564]]
(2L, 10L)


In [39]:
import nltk
import pprint
import numpy as np
from keras.preprocessing import sequence
def get_sents(string, vocab, doc_len=100, sent_len=50):
    """
    :param string:
    :return: ndarray:shape=(doc_len, sent_len), mask: shape=(doc_len, sent_len), num_hit, unk_hit, total
    """

    num_hit = 0
    unk_hit = 0
    total = 0
    sents = nltk.sent_tokenize(string)
    doc_matrix = np.zeros(shape=(doc_len, sent_len))
    for i, sent in enumerate(sents):
        tokens = nltk.word_tokenize(sent)
        indices = []
        if i>= doc_len:
            break
        for word in tokens:
            indices.append(1)
            total += 1    
        doc_matrix[i,:] = sequence.pad_sequences([indices], sent_len, padding='post')
    return doc_matrix, num_hit, unk_hit, total
x="@ORGANIZATION1, @NUM1 out of @NUM2 people surveyed in @LOCATION1, @CAPS1 agree that the use of computers benefits society. I personally concer" \
             "n with these @NUM1 people, computers have a positive effect on people."
# sents = nltk.sent_tokenize(x)
print get_sents(x,{'out':1, 'people':2})[0].shape

(100L, 50L)


[[2, 3, 4], [1, 2, 3]]
